In [1]:
%matplotlib
import numpy as np
import matplotlib.pyplot as plt

Using matplotlib backend: Qt5Agg


In [95]:
def err_cnt(dataset):
    data=np.mat(dataset)
    return np.var(data[:,-1])*np.shape(data)[0]    #总方差
#l=err_cnt([[1,1,1],
#        [1,2,0],
#        [1,2,2]])

In [126]:
def split_tree(dataset,feats,value):
    set_1=[]
    set_2=[]
    for x in dataset:
        if  x[feats]>=value:
            set_2.append(x)
        else:
            set_1.append(x)
    return (set_1,set_2)

In [140]:
class node:
    def __init__(self,feats=1,value=None,results=None,right=None,left=None):
        self.feats=feats
        self.value=value
        self.results=results
        self.left=left
        self.right=right

In [170]:
def buildtree(data,minsample,minerr):
    if len(data)<minsample:
        return node(results=leaf(data))
    
    best_err=err_cnt(data)
    best_Criteria=None
    bestSets=None
    
    featue_num=len(data[0])-1
    for feats in range(featue_num):
        feature_values={}
        for sample in data:
            feature_values[sample[feats]]=1
        for value in feature_values.keys():
            (set_1,set_2)=split_tree(data,feats,value)
            if len(set_1)<2 or len(set_2)<2:
                continue;
            now_err=err_cnt(set_1)+err_cnt(set_2)
            if now_err< best_err and len(set_1)>0 and len(set_2)>0:
                best_err=now_err
                best_Criteria=(feats,value)
                bestSets=(set_1,set_2)
    if best_err>minerr:
        right=buildtree(bestSets[1],minsample,minerr)
        left=buildtree(bestSets[0],minsample,minerr)
        return node(feats=best_Criteria[0],value=best_Criteria[1],right=right,left=left)
    else:
        return node(results=leaf(data))

In [151]:
def leaf(dataset):
    dataset=np.array(dataset)
    data=np.mat(dataset)
    return np.mean(dataset[:,-1])

In [177]:
import pickle
if __name__=="__main__":
    print("load data-------")
    data=load_data("sine.txt")
    print("build tree------")
    regression_tree=buildtree(data,30,0.3)
    print("cal err----------")
    err=cal_err(data,regression_tree)
    print("save tree---------")
    save_model(regression_tree,"regression_tree")

load data-------
build tree------
cal err----------
0.6801104289924979
save tree---------


In [93]:
def load_data(datafile):
    data=[]
    f=open(datafile)
    for line in f.readlines():
        sample=[]
        lines=line.strip().split("\t")
        for x in lines:
            sample.append(float(x))
        data.append(sample)
    f.close()
    return data

In [169]:
def cal_err(data,tree):
    m=len(data)
    n=len(data[0])-1
    err=0.0
    for i in range(m):
        temp=[]
        for j in range(n):
            temp.append(data[i][j])
        pre=predict(temp,tree)
        err+=(data[i][j]-pre)**2
    return err/m

In [83]:
def save_model(tree,file):
    with open(file,'wb') as f:
        pickle.dump(tree,f)

In [174]:
def predict(sample,tree):
    if tree.results !=None:
        return tree.results
    else:
        val_sample=sample[tree.feats]
        if val_sample>=tree.value:
            branch=tree.right
        else:
            branch=tree.left
    return predict(sample,branch)